![img](https://github.com/JuliaLang/julia/raw/master/doc/src/assets/logo.svg)![img](https://avatars.githubusercontent.com/u/7346142?s=200&v=4)

# Installation and Setup

JuliaGPU packages are easy to install: Just do `Pkg.add("CUDA")` to install the CUDA.jl package, which provides bindings to NVIDIA's CUDA. CUDA.jl provides all of the compiler and runtime logic needed to program NVIDIA GPUs; the only thing you need to provide is a functional NVIDIA driver (which most HPC systems already have installed and configured), but you don't need to install the CUDA toolkit! CUDA.jl downloads one if it's not already available on your system (again, HPC systems usually have this provided for you):

In [ ]:
# This can take a little while to download and compile, so just be patient
import Pkg
Pkg.add("CUDA")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


We're going to be running some small benchmarks in this notebook, so let's also grab Julia's BenchmarkTools.jl while we're at it:

In [ ]:
import Pkg
Pkg.add("BenchmarkTools")

   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile
Precompiling packages...
   1930.1 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 3 seconds. 460 already precompiled.


And now we import these packages, along with the built-in LinearAlgebra standard library:

In [ ]:
using CUDA
using BenchmarkTools

using LinearAlgebra

Now, GPU vendor libraries can be difficult, so CUDA.jl provides a convenient way to check if everything is setup correctly, the `CUDA.versioninfo()` function. Like Julia's `Base.versioninfo()`, this will print some information on the available hardware and loaded libraries:

In [ ]:
CUDA.versioninfo()

CUDA runtime 12.5, local installation
CUDA driver 12.8
NVIDIA driver 550.54.15

CUDA libraries: 
- CUBLAS: 12.5.3
- CURAND: 10.3.6
- CUFFT: 11.2.3
- CUSOLVER: 11.6.3
- CUSPARSE: 12.5.1
- CUPTI: 2024.2.1 (API 23.0.0)
- NVML: 12.0.0+550.54.15

Julia packages: 
- CUDA: 5.7.2
- CUDA_Driver_jll: 0.12.1+1
- CUDA_Runtime_jll: 0.16.1+0
- CUDA_Runtime_Discovery: 0.3.5

Toolchain:
- Julia: 1.10.9
- LLVM: 15.0.7

Preferences:
- CUDA_Runtime_jll.version: 12.5.1
- CUDA_Runtime_jll.local: true

1 device:
  0: NVIDIA A100-SXM4-40GB (sm_80, 39.552 GiB / 40.000 GiB available)


It's always good practice to check this at least once on a new system or when you mess with `module`s loaded, just to ensure that everything is connected and happy!

# Example: Vector Addition

As a simple example, let's take a look at vector addition. Let's assume you have two vectors $\vec{a}$ and $\vec{b}$ and you want to add them elementwise. You can do this in many ways in Julia:
1. simple for loop on a CPU
2. julia array add (+) on a CPU or GPU
3. GPU kernel programming in CUDA (or KernelAbstractions using CUDA as backend - we'll see this soon!)

In [ ]:
# define our input a, b vectors, and output c vector in CPU RAM
vector_size = 1024
a = rand(1:4, vector_size)
b = rand(1:4, vector_size)
c = zeros(Int, vector_size)

# what's in a?
a

1024-element Vector{Int64}:
 3
 4
 4
 3
 3
 1
 4
 2
 1
 3
 3
 1
 3
 ⋮
 4
 4
 1
 3
 1
 2
 2
 3
 2
 4
 1
 2

Let's write a simple CPU loop to add two vectors in serial:

In [ ]:
# Note: the exclamation mark (!) doesn't do anything special
# It's just used to indicate that a function mutates its arguments
function vadd!(a, b, c)
    for i in 1:length(c)
        c[i] = a[i] + b[i]
    end
end
vadd!(a, b, c)
c

1024-element Vector{Int64}:
 7
 8
 8
 7
 4
 5
 6
 4
 3
 6
 5
 2
 7
 ⋮
 6
 7
 4
 6
 4
 3
 4
 5
 5
 8
 4
 6

Thankfully, Julia has a ton of built-in array operations, so we don't actually need to implement this ourselves. Julia's vector add (+) operation works exactly as you'd expect:

In [ ]:
c = a + b

# Note that, unlike `vadd!`, the above operation allocates a new `c` as the output vector - this is important to remember.

1024-element Vector{Int64}:
 7
 8
 8
 7
 4
 5
 6
 4
 3
 6
 5
 2
 7
 ⋮
 6
 7
 4
 6
 4
 3
 4
 5
 5
 8
 4
 6

Great! But isn't this a GPU tutorial? Let's get to it!

In [ ]:
# We need first to make copies of the a and b vectors on the GPU, and define a new dc empty GPU vector

# The CuArray() function automatically allocates a new GPU array of the same size and shape as the input,
# and copies from the input CPU array to the newly-allocated GPU array
da = CuArray(a)
db = CuArray(b)

# CUDA.zeros takes the desired element type and array size, and automatically allocates and initializes
# a new GPU array with int64 zeros
dc = CUDA.zeros(Int, size(a))

# We can also safely take a look at what's in da, even though it's on the GPU!
# It's a different array type, and that fact is made clear to us:
da

1024-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 3
 4
 4
 3
 3
 1
 4
 2
 1
 3
 3
 1
 3
 ⋮
 4
 4
 1
 3
 1
 2
 2
 3
 2
 4
 1
 2

Now that we know how to allocate on the GPU, let's see how to use this same add (+) operation on the GPU to add two of those vectors using CUDA:

In [ ]:
dc = da + db

# We can add GPU vectors using the same `+` operator, thanks to Julia's multiple dispatch!
# Also, like for the CPU add operation, this one also allocates a new GPU array for output `dc`

1024-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 7
 8
 8
 7
 4
 5
 6
 4
 3
 6
 5
 2
 7
 ⋮
 6
 7
 4
 6
 4
 3
 4
 5
 5
 8
 4
 6

Cool, but vector addition is pretty... simple? Let us learn how to write our own GPU kernels with CUDA.jl in pure Julia.

In array operations, CUDA.jl can leverage implicit parallelism (expressed over the array's elements) to automatically execute these operations in parallel on a GPU. When using hand-rolled kernels, it is instead the programmer's responsibility to decide how to effectively assign the available parallel execution resources for the specific operation. Let's see how this is done for vector addition, before moving on to more interesting examples:

In [ ]:
function vadd_kernel!(c, a, b)
    # Obtain GPU thread index, which should be mapped to the valid indices of a and b
    i = threadIdx().x
    # Each thread will add its own element to c
    c[i] = a[i] + b[i]

    # GPU kernels don't return anything
    return
end

vadd_kernel! (generic function with 1 method)

At a high level, that's pretty easy, you just need to write a scalar function, just like you'd do if you were writing CUDA C++. Now we just need to launch that function in parallel using the `@cuda` macro, and specify the number of GPU threads with the `threads` keyword argument:

In [ ]:
# Launch our `vadd_kernel!` GPU kernel, with our GPU arrays as inputs
@cuda threads=length(da) vadd_kernel!(dc, da, db)

dc

1024-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 7
 8
 8
 7
 4
 5
 6
 4
 3
 6
 5
 2
 7
 ⋮
 6
 7
 4
 6
 4
 3
 4
 5
 5
 8
 4
 6

OK, this is great, and was not a lot of work for us! But to see a downside of this simple approach, let's try to work with bigger vectors, by setting `vector_size` to 10240:

In [ ]:
vector_size = 10240
da = CuArray(rand(1:4, vector_size))
db = CuArray(rand(1:4, vector_size))
dc = CUDA.zeros(Int, vector_size)

@cuda threads=length(da) vadd_kernel!(dc, da, db)

LoadError: Number of threads in x-dimension exceeds device limit (10240 > 1024).

Oh no! What is going on here?

GPUs have a limited number of threads they can run on a single streaming multiprocessor (SM) at once, and we just tried to assign too many threads to one SM, which isn't possible:

In [ ]:
# To query the number of threads per block, we can inspect CUDA attributes:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)

1024

Since 10240 > 1024, the SM wouldn't have had enough resources to satisfy our request, at least not with the default of a single block per kernel.

Thankfully, GPUs also have multiple SMs, so in theory this should be solvable. To take advantage of more than one SM, we need to run a kernel with multiple blocks, as a single block can only execute on one SM (which has limited resources available, as we saw in the query above). In order to exploit multiple blocks, though, we need to understand how to index into our arrays when our index depends not just on our thread index, but also on our block index and block sizes.

In CUDA.jl, the expression `i = threadIdx().x + (blockIdx().x - 1) * blockDim().x` calculates a unique index for each thread across multiple blocks in a CUDA kernel execution. Here's a breakdown of each component and how they contribute to computing this index:

- `threadIdx().x`: This returns the x-coordinate of the thread within its block. It's the thread's index within the block, starting from 1 (unlike C/C++ CUDA where it starts from 0).

- `blockIdx().x`: This gives the x-coordinate of the block within the grid. It represents the block's index in the grid, also starting from 1.

- `blockDim().x`: This represents the number of threads per block along the x-axis.

In essence, the formula `i = threadIdx().x + (blockIdx().x - 1) * blockDim().x` is used to compute a global index for each thread, regardless of how blocks are sized. It positions the threads linearly across all blocks. Here's what each part does:

- `(blockIdx().x - 1) * blockDim().x`: This part calculates the offset to the start of the current block. Subtracting 1 from `blockIdx().x` makes it zero-based, and then it is multiplied by the number of threads in each block `(blockDim().x)`. This gives the index of the first thread in the current block relative to the entire grid.

- `threadIdx().x`: Adding this to the block offset gives the specific thread's index within the whole grid.

Knowing this, let's now rewrite our kernel to properly handle multiple blocks, using the above global indexing formula:

In [ ]:
function vadd_cuda!(c, a, b)
    # Calculate a unique index for each thread across multiple blocks
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x

    # Ensure that we skip invalid indices, if we over-allocated a few threads
    if i <= length(a)
        c[i] = a[i] + b[i]
    end

    return
end

vadd_cuda! (generic function with 1 method)

Now we can launch our kernel with the maximum number of threads per block (1024), and then divide up our computation across multiple 1024-wide blocks:

In [ ]:
@cuda threads=1024 blocks=cld(length(da),1024) vadd_cuda!(dc, da, db) # cld(x, y) is (x / y) with round-up behavior
dc

10240-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 4
 4
 3
 5
 7
 5
 4
 7
 4
 2
 8
 5
 5
 ⋮
 5
 8
 4
 5
 5
 3
 8
 3
 4
 6
 3
 6

Yay! Now that we're thoroughly done with vector addition, let's move on to something a bit heavier:

## Example: Matrix-Matrix Multiplication

Matrix multiplication is a mainstay of all kinds of applications, so we should be able to implement this in Julia with ease. Let's first take a look at doing this on the CPU:

In [ ]:
# Allocate our random matrix inputs and zero'd output
matrix_size = 1024
A = rand(matrix_size, matrix_size)
B = rand(matrix_size, matrix_size)
C = zeros(matrix_size, matrix_size)

A

1024×1024 Matrix{Float64}:
 0.111411  0.0846486  0.0497382  …  0.613677     0.377845    0.0341445
 0.174748  0.350421   0.227078      0.671424     0.141305    0.147167
 0.731431  0.185005   0.503551      0.148479     0.880455    0.0922076
 0.690734  0.108755   0.705794      0.825014     0.0326488   0.471357
 0.84956   0.149493   0.163321      0.71118      0.736062    0.747991
 0.785406  0.270562   0.0286078  …  0.903958     0.225837    0.322484
 0.33145   0.667579   0.498192      0.442454     0.24898     0.910926
 0.763405  0.352141   0.958001      0.000287206  0.353318    0.513929
 0.684119  0.0739164  0.41486       0.47827      0.875003    0.61574
 0.529939  0.792093   0.336086      0.595774     0.529696    0.320064
 0.99392   0.917188   0.309304   …  0.230307     0.189118    0.443715
 0.250578  0.451906   0.962513      0.174126     0.885104    0.569134
 0.946768  0.228864   0.445785      0.279041     0.00986295  0.569224
 ⋮                               ⋱                           


The three nested loops implementation of matrix multiplication is easy to express on the CPU:

In [ ]:
function MatrixMultiplication!(C, A, B)
    for i in 1:size(C, 1)
        for j in 1:size(C, 2)
            C[i, j] = 0
            for k in 1:size(A, 2)
                C[i, j] += A[i, k] * B[k, j]
            end
        end
    end
end


MatrixMultiplication! (generic function with 1 method)

In [ ]:
MatrixMultiplication!(C, A, B)
C

1024×1024 Matrix{Float64}:
 245.013  240.989  248.09   251.864  …  243.379  252.709  246.208  245.343
 248.218  247.06   247.529  250.108     247.16   260.909  252.556  245.11
 259.227  255.325  253.269  256.752     257.718  264.649  258.87   251.411
 254.971  254.935  252.419  257.587     257.727  264.72   256.444  245.325
 252.853  250.352  249.558  248.129     250.415  254.944  256.359  245.326
 243.896  242.375  239.664  242.608  …  242.506  248.201  250.104  238.062
 244.934  240.909  246.583  249.87      248.93   254.613  252.11   241.863
 255.599  254.366  259.545  255.809     255.401  263.658  257.591  249.531
 254.695  245.282  250.888  249.315     243.945  256.361  255.367  245.544
 252.042  255.34   249.833  250.79      250.995  255.279  254.641  246.194
 257.263  254.936  260.803  264.896  …  262.311  264.158  263.657  252.691
 253.08   248.461  247.256  252.968     251.76   258.748  254.374  242.948
 253.739  253.971  249.772  250.995     254.165  263.794  256.429  244.85


Of course, Julia has this one built-in already (it calls BLAS):

In [ ]:
C = A * B

1024×1024 Matrix{Float64}:
 245.013  240.989  248.09   251.864  …  243.379  252.709  246.208  245.343
 248.218  247.06   247.529  250.108     247.16   260.909  252.556  245.11
 259.227  255.325  253.269  256.752     257.718  264.649  258.87   251.411
 254.971  254.935  252.419  257.587     257.727  264.72   256.444  245.325
 252.853  250.352  249.558  248.129     250.415  254.944  256.359  245.326
 243.896  242.375  239.664  242.608  …  242.506  248.201  250.104  238.062
 244.934  240.909  246.583  249.87      248.93   254.613  252.11   241.863
 255.599  254.366  259.545  255.809     255.401  263.658  257.591  249.531
 254.695  245.282  250.888  249.315     243.945  256.361  255.367  245.544
 252.042  255.34   249.833  250.79      250.995  255.279  254.641  246.194
 257.263  254.936  260.803  264.896  …  262.311  264.158  263.657  252.691
 253.08   248.461  247.256  252.968     251.76   258.748  254.374  242.948
 253.739  253.971  249.772  250.995     254.165  263.794  256.429  244.85


Our implementation performs quite a bit worse than Julia's (OpenBLAS), but that's OK - we haven't really optimized it at all, since this is just a tutorial:

In [ ]:
@benchmark MatrixMultiplication!(C, A, B)

BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  3.819 s …   3.872 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.845 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.845 s ± 37.426 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.82 s         Histogram: frequency by time        3.87 s <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [ ]:
@benchmark A * B

BenchmarkTools.Trial: 597 samples with 1 evaluation per sample.
 Range (min … max):  6.956 ms … 16.250 ms  ┊ GC (min … max): 0.00% … 7.90%
 Time  (median):     7.223 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.351 ms ±  1.920 ms  ┊ GC (mean ± σ):  1.74% ± 4.95%

  ▂█▅▁                ▁         ▂▂                            
  ████▇█▅▆▅▁▄▄▅▁▅▁▅▄▄▄█▇▄▄▅▁▆▄████▅▆█▆▅▅██▅▁▁▁▁▁▁▄▅▁▄▄▁▁▄▁▁▄ ▆
  6.96 ms      Histogram: log(frequency) by time     14.6 ms <

 Memory estimate: 8.00 MiB, allocs estimate: 2.

Ok, let's now implement matrix multiplication on the GPU:

In [ ]:
# We need first to move A and B matrices to the GPU and define a new DC zero'd matrix on the GPU
DA = CuArray(A)
DB = CuArray(B)
DC = CUDA.zeros(size(A))

DA

1024×1024 CuArray{Float64, 2, CUDA.DeviceMemory}:
 0.111411  0.0846486  0.0497382  …  0.613677     0.377845    0.0341445
 0.174748  0.350421   0.227078      0.671424     0.141305    0.147167
 0.731431  0.185005   0.503551      0.148479     0.880455    0.0922076
 0.690734  0.108755   0.705794      0.825014     0.0326488   0.471357
 0.84956   0.149493   0.163321      0.71118      0.736062    0.747991
 0.785406  0.270562   0.0286078  …  0.903958     0.225837    0.322484
 0.33145   0.667579   0.498192      0.442454     0.24898     0.910926
 0.763405  0.352141   0.958001      0.000287206  0.353318    0.513929
 0.684119  0.0739164  0.41486       0.47827      0.875003    0.61574
 0.529939  0.792093   0.336086      0.595774     0.529696    0.320064
 0.99392   0.917188   0.309304   …  0.230307     0.189118    0.443715
 0.250578  0.451906   0.962513      0.174126     0.885104    0.569134
 0.946768  0.228864   0.445785      0.279041     0.00986295  0.569224
 ⋮                               ⋱     

In the same way, here we can multiply the `DA` matrix by `DB` matrix using the `*` operator (which forwards the call to CUBLAS), thanks again to Julia's multiple dispatch:

In [ ]:
DC = DA * DB

1024×1024 CuArray{Float64, 2, CUDA.DeviceMemory}:
 245.013  240.989  248.09   251.864  …  243.379  252.709  246.208  245.343
 248.218  247.06   247.529  250.108     247.16   260.909  252.556  245.11
 259.227  255.325  253.269  256.752     257.718  264.649  258.87   251.411
 254.971  254.935  252.419  257.587     257.727  264.72   256.444  245.325
 252.853  250.352  249.558  248.129     250.415  254.944  256.359  245.326
 243.896  242.375  239.664  242.608  …  242.506  248.201  250.104  238.062
 244.934  240.909  246.583  249.87      248.93   254.613  252.11   241.863
 255.599  254.366  259.545  255.809     255.401  263.658  257.591  249.531
 254.695  245.282  250.888  249.315     243.945  256.361  255.367  245.544
 252.042  255.34   249.833  250.79      250.995  255.279  254.641  246.194
 257.263  254.936  260.803  264.896  …  262.311  264.158  263.657  252.691
 253.08   248.461  247.256  252.968     251.76   258.748  254.374  242.948
 253.739  253.971  249.772  250.995     254.165  26

In [ ]:
function MatrixMultiplication_cuda!(C, A, B)
    # Calculate the global row and column indices
    row = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    col = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Create a 0 of the same type as C's element type (for type stability)
    sum = zero(eltype(C))

    if row <= size(A, 1) && col < size(B, 2)
        for i in 1:size(A, 2)
            # @inbounds disables bounds checking for array accesses, to improve performance
            # Note that incorrect usage can result in segfaults/memory faults/wrong results
            @inbounds sum += A[row, i] * B[i, col]
        end
        C[row, col] = sum
    end

    return
end

MatrixMultiplication_cuda! (generic function with 1 method)

In [ ]:
# Split blocks up into 32x32 tiles
@cuda threads=(32, 32) blocks=(matrix_size ÷ 32, matrix_size ÷ 32) MatrixMultiplication_cuda!(DC, DA, DB)

DC

1024×1024 CuArray{Float64, 2, CUDA.DeviceMemory}:
 245.013  240.989  248.09   251.864  …  243.379  252.709  246.208  245.343
 248.218  247.06   247.529  250.108     247.16   260.909  252.556  245.11
 259.227  255.325  253.269  256.752     257.718  264.649  258.87   251.411
 254.971  254.935  252.419  257.587     257.727  264.72   256.444  245.325
 252.853  250.352  249.558  248.129     250.415  254.944  256.359  245.326
 243.896  242.375  239.664  242.608  …  242.506  248.201  250.104  238.062
 244.934  240.909  246.583  249.87      248.93   254.613  252.11   241.863
 255.599  254.366  259.545  255.809     255.401  263.658  257.591  249.531
 254.695  245.282  250.888  249.315     243.945  256.361  255.367  245.544
 252.042  255.34   249.833  250.79      250.995  255.279  254.641  246.194
 257.263  254.936  260.803  264.896  …  262.311  264.158  263.657  252.691
 253.08   248.461  247.256  252.968     251.76   258.748  254.374  242.948
 253.739  253.971  249.772  250.995     254.165  26

And as we'd expect, the optimized CUBLAS implementation is far faster than ours:

In [ ]:
@benchmark DA * DB

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):   38.058 μs …  22.594 ms  ┊ GC (min … max): 0.00% … 23.21%
 Time  (median):     130.422 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   126.368 μs ± 316.869 μs  ┊ GC (mean ± σ):  0.83% ±  0.33%

  ▅▃▁                                            ▁▂▂▂▄▆█▇▆▄▃▂▂▁ ▂
  ███▇▇▆▆▆▆▆▅▄▄▃▃▁▃▁▃▃▃▃▃▃▄▁▁▁▃▁▁▃▄▄▃▁▁▃▄▄▅▄▆▆▆▇███████████████ █
  38.1 μs       Histogram: log(frequency) by time        143 μs <

 Memory estimate: 1.62 KiB, allocs estimate: 71.

In [ ]:
@benchmark CUDA.@sync @cuda threads=(32, 32) blocks=(matrix_size ÷ 32, matrix_size ÷ 32) MatrixMultiplication_cuda!(DC, DA, DB)

BenchmarkTools.Trial: 4039 samples with 1 evaluation per sample.
 Range (min … max):  1.214 ms …  1.329 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.228 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.231 ms ± 10.203 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▂▃▇▆▇█▇▅▅▃▄▂▁▂                                       
  ▁▁▁▃▄▆▇██████████████▇▆▆▅▅▅▅▄▃▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.21 ms        Histogram: frequency by time        1.27 ms <

 Memory estimate: 1.27 KiB, allocs estimate: 36.

Ouch! Why exactly is our kernel implementation slower?

The answer is that this is only the naive implementation of matrix multiplication - if you did the same in CUDA C++, you'd get similarly bad performance. The performant implementation relies on tiling, where the matrix is divided into smaller submatrices (tiles) that fit more effectively within the GPU’s memory hierarchy, including shared memory and cache. Additionally, optimized kernels would use shared memory and WMMA instructions to greatly improve data locality and reduce bandwidth needs (both of which can be easily used in Julia).

In an optimized implementation, each thread block on the GPU handles a specific tile of the output matrix, loading portions of the input tiles into shared memory to reduce the repeated global memory access. This approach enables a higher level of parallelism by allowing multiple tiles to be processed concurrently across the GPU cores, without bottlenecking on memory transfers.

For the purpose of this tutorial, we will not be implementing these optimizations, but do know that they are as easy (or easier) to use in Julia compared to CUDA C++.

# KernelAbstractions

Now that we know how to write vendor-specific kernels, let's explore the naive matrix multiplication example using KernelAbstractions.jl:

In [ ]:
import Pkg
Pkg.add("KernelAbstractions")

   Resolving package versions...
  No Changes to `/global/u2/t/train921/julia-hpc-tutorial-sc24-main/Project.toml`
  No Changes to `/global/u2/t/train921/julia-hpc-tutorial-sc24-main/Manifest.toml`


We'll also load the Random standard library to assist with certain random array initializations:

In [ ]:
using KernelAbstractions
using Random

Implementing a kernel with KernelAbstractions is very similar to implementing a kernel with CUDA.jl. The primary differences include annotating a kernel function with `@kernel`, and doing thread indexing using `@index` (which efficiently abstracts away the index calculations we were previously doing). Otherwise, most things are the same:

In [ ]:
@kernel function MatrixMultiplication_kernel!(C, A, B)
    # Global index of each thread across multiple blocks in both x and y dimensions of the grid
    row, col = @index(Global, NTuple)

    # Everything else is the same!
    sum = zero(eltype(C))

    if row <= size(A, 1) && col <= size(B, 2)
        for i = 1:size(A, 2)
             @inbounds sum += A[row, i] * B[i, col]
        end
        @inbounds C[row, col] = sum
     end
end

MatrixMultiplication_kernel! (generic function with 4 methods)

One key difference between KernelAbstractions and CUDA is that, because KernelAbstractions is portable, we need to select the CUDA "backend" when we compile our kernel (AMDGPU, Apple, and Intel are also supported). Most operations take the backend as the first argument, to allow Julia's multiple dispatch to redirect calls to the correct implementation. Additionally, KernelAbstractions separate the compilation and kernel launch stages, and provides configurations for each step to optimize further.

In [ ]:
# Select the CUDA backend
Backend = CUDA.CUDABackend()

# Use KernelAbstractions's APIs to allocate GPU matrices DA, DB, and DC
matrix_size = 2048
T = Float64
DA = rand!(allocate(Backend, T, matrix_size, matrix_size))
DB = rand!(allocate(Backend, T, matrix_size, matrix_size))
DC = KernelAbstractions.zeros(Backend, T, matrix_size, matrix_size)

# Compile the kernel
# We'll statically assign the workgroup (AKA block) size to allow for additional compile-time optimizations
workgroupsize = (32, 32)
kernel! = MatrixMultiplication_kernel!(Backend, workgroupsize)

# Launch the kernel with our GPU matrices as inputs
kernel!(DC, DA, DB, ndrange=(size(DC)))

# Explicitly wait for the kernel to complete
KernelAbstractions.synchronize(Backend)

# Are our results what we'd expect to see (compared to CUBLAS)?
isapprox(DC, DA * DB)

true

In [ ]:
@benchmark begin
    kernel!(DC, DA, DB, ndrange=size(DC))
    KernelAbstractions.synchronize(Backend)
end

BenchmarkTools.Trial: 348 samples with 1 evaluation.
 Range (min … max):  13.950 ms … 14.059 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.013 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.012 ms ± 20.283 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                        ▁ ▃▃▂▅  ▄▁▂▆▅▃▄▂▄▄▁█▄▂█▄▂              
  ▄▁▁▃▁▃▃▃▁▄▅▅▄▆▁▃▃▅▅▄▆▅█▆█████▇█████████████████▇▅▆█▇▆▇▃▅▄▅▅ ▅
  14 ms           Histogram: frequency by time        14.1 ms <

 Memory estimate: 1.89 KiB, allocs estimate: 60.

# Example: Memory copy with KernelAbstractions

Let's now see a different kind of example, to show how to use shared memory in KernelAbstractions. This kernel performs a matrix copy using local memory (also known as shared memory in CUDA), which can significantly speed up the memory access times by reducing global memory bandwidth usage:

In [ ]:
@kernel function lmem_copy_kernel!(output, @Const(input))
    # Gets the global index of the thread in a multidimensional grid, which is used to index into the global input and output arrays.
    I, J = @index(Global, NTuple)
    # Gets the local index within a thread block or workgroup, useful for indexing into locally shared memory.
    i, j = @index(Local, NTuple) # Local index of thread

    # @groupsize() retrieves the dimensions of the thread block or workgroup.
    # The @uniform ensures that these values are treated as constants that are the same for all threads.
    N = @uniform @groupsize()[1] # same as blockDim().x
    M = @uniform @groupsize()[2] # same as blockDim().y

    # Allocate local (shared) memory
    tile = @localmem eltype(output) (N, M)

    # First, data from the global input array is loaded into the shared tile array using local indices.
    @inbounds tile[i, j] = input[I, J]

    # @synchronize ensures that all threads in the workgroup have completed their memory writes to the shared memory before proceeding.
    # This is crucial to prevent race conditions.
    @synchronize

    # Finally, the data is written back from the shared tile array to the global output array.
    @inbounds output[I, J] = tile[i, j]
end

lmem_copy_kernel! (generic function with 4 methods)

This kernel is a little bit more "advanced" than prior kernels, but still is quite readable once you know what each of the macros do. We can quickly test that it works correctly:

In [ ]:
# Allocate inputs and outputs
input = rand!(allocate(Backend, T, matrix_size, matrix_size))
output = KernelAbstractions.zeros(Backend, T, matrix_size, matrix_size)

# Compile and launch the kernel, and wait for it to complete
lmem_copy! = lmem_copy_kernel!(Backend, workgroupsize)
lmem_copy!(output, input, ndrange=size(input))
KernelAbstractions.synchronize(Backend)

# Confirm that the output matrix now matches the input matrix
all(input == output)

true